## 게이트가 추가된 RNN
- RNN은 순환 경로를 포함하여 과거의 정보를 기억할 수 있다.
- RNN의 단점은 시계열 데이터에서 시간적으로 멀리 떨어진 long term dependency는 잘 학습할 수 없다는데 있다.
- LSTM이나 GRU는 Gate라는 구조가 더해진다, 이 게이트 덕분에 시계열 데이터의 Long term dependency를 학습할 수 있다.

## 6.1 RNN의 문제점
- 앞서 설명한 RNN은 시계열 데이터의 장기 의존 관계를 학습하기 어렵다.
- 그 원인은 Backproap 시 깊이가 깊어져 Vanishing Gradient 문제가 발생한다.

#### 6.1.1 RNN 복습

![image](https://github.com/choibigo/Study/assets/38881179/f3fd1ebc-36aa-4cbe-9bd0-a6548fa5df61)

- RNN 계층은 시계열 데이터인 xt를 입력하면 ht를 출력한다.
- ht는 RNN 계층의 은닉 상태 라고 하며, 과거 정보를 저장한다.
- RNN의 특징은 바로 이전 시각의 은닉 상태를 이용한다는 점이다.

#### 6.1.2 기울기 손실 또는 기울기 폭발
- "Tom was watching TV in his room. Mary came into the room. Mary siad hi to [ ? ]
- [ ? ]에 들어가는 산어는 Tom 이다. 앞에 있는 모든 문장을 확인하여 [ ? ]을 파악해야 한다.

![image](https://github.com/choibigo/Study/assets/38881179/bbeca224-8fde-4a34-a922-b69390b4b567)

- RNN 계층이 과거 방향으로 '의미 있는 기울기'를 전달함으로써 시간 방향의 의존 관계를 학습할 수 있다.
- 원래대로라면 기울기에 학습해야 할 의미가 있는 정보가 들어 있고, 그것을 과거로 전달함으로써 장기 의존 관계를 학습한다.
- 하지만 이 기울기가 없어진다면 아무런 정보도 남지 않고 Weight는 갱신되지 않는다. (Vanishing Gradient)
- 단순한 RNN 계층은 Vanishing Gradient나 Exploding Gradient 가 나타난다.

#### 6.1.3 Vanishing & Exploding Gradient

![image](https://github.com/choibigo/Study/assets/38881179/dbf97421-8744-4aef-a74a-446264f552a6)

- T번째 정답 레이블이 "Tom"인 경우에 해당한다.
- 시간 방향 기울기에 주목하면 역전파로 전해지는 기울기는 차례대로 'tanh', '+', 'MatMul' 연산을 통과하는 것을 알 수 있다.
- '+'의 역전파는 상류에서 전해지는 기울기를 그대로 하류로 흘려 보낸다, 그래서 기울기는 변하지 않는다.
- y = tanh(x)일 떄의 미분은 ∂y/∂x = 1-y^2이다, 이때 y = tanh(x)의 값과 그 민분값을 그래프로 표현하면 아래와 같다.

![image](https://github.com/choibigo/Study/assets/38881179/f20fab89-ba87-4cc5-aa1d-604ec2e326b0)

- tanh의 미분값은 0~1 사이 값이다, x가 0으로 부터 멀어지면 미분값(기울기)가 점점 작아진다.
- tanh 함수를 T번 통과하면 기울기도 T번 곱해져서 점점 작아지게 된다.

![image](https://github.com/choibigo/Study/assets/38881179/c6ddd6fa-57dd-4612-9c48-7a0bff7948a7)

- MatMul의 역전파만 파악하기 위해 tanh를 생략한다.
- dh라는 기울기가 흘러 들어오면 Matmul 노드에서 역전파는 dhWh^t라는 생렬 곱으로 기울기를 계산한다.
- 같은 계산을 시계열 데이터의 시간 크기 만큼 반복한다, 이때 행렬 곱셈 에서는 매번 같은 가중치 Wh가 사용된다.
- Wh 그림은 따로 그려져 있지만 이전 출력에 계속 곱해지는 같은 값이다.

#### MatMul의 역전파


In [1]:
import numpy as np
from matplotlib.pyplot import plot

np.random.seed(3)

N = 2
H = 3
T = 20

dh = np.ones((N, H)) # 첫 dh는 ones 행렬로 만듬
Wh = np.random.randn(H, H) # Wh는 랜덤하게 만듬 (실제로는 학습 되는 Weight 값이다.)

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T) #역전파된 값이다. dh 와 Wh의 transpose를 행렬곱
    norm = np.sqrt(np.sum(dh**2)) / N # 역전파의 2-Norm 을 구한다.
    norm_list.append(norm)

print(norm_list)

[2.4684068094579303, 3.3357049741610365, 4.783279375373182, 6.279587332087612, 8.080776465019053, 10.251163032292936, 12.936063506609896, 16.276861327786712, 20.45482961834598, 25.688972842084684, 32.25315718048336, 40.48895641683869, 50.8244073070191, 63.79612654485427, 80.07737014308985, 100.5129892205125, 126.16331847536823, 158.35920648258823, 198.7710796761195, 249.495615421267]


- 기울기는 시간에 비례해 지수적으로 증가한다, 이것이 바로 Exploding Gradient이다.
- 이러한 Exploding Gradient가 일어나면 오버플로우가 일어나고 NaN(Not a Number)을 발생한다.

In [3]:
import numpy as np
from matplotlib.pyplot import plot

np.random.seed(3)

N = 2
H = 3
T = 20

dh = np.ones((N, H))
Wh = np.random.randn(H, H)  * 0.5

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T)
    norm = np.sqrt(np.sum(dh**2)) / N
    norm_list.append(norm)

print(norm_list)

[1.2342034047289652, 0.8339262435402591, 0.5979099219216477, 0.39247420825547574, 0.2525242645318454, 0.16017442237957713, 0.10106299614538981, 0.06358148956166684, 0.03995083909833199, 0.025086887541098325, 0.015748611904532892, 0.009884999125204758, 0.006204151282595105, 0.003893806551809953, 0.002443767399386287, 0.0015337065005571365, 0.0009625497320203265, 0.0006040924319556741, 0.00037912574706291106, 0.00023793756048323344]


- 기울기가 지수적으로 감소한다, 이것이 Vanishing Gradient이다.
- 기울기가 매우 작아져서 0과 가까워 진다면 Optmizer가 제대로 작동하지 않아 Weight가 갱신되지 않는다.
- 따라서 앞쪽 Layer의 초기 Weight(의미 없이 random인 데이터)가 그대로 유지되어 네트워크가 전체적으로 학습되지 않는다.
- Wh가 T번 반복해서 곱해진다. => N제곱 된다 => 지수적으로 작아지거나 커진다.

#### 6.1.4 기울기 폭발 대책
- 기울기 폭발의 대책으로 Gradient Clipping이라는 전통적인 기법이 있다.

![image](https://github.com/choibigo/Study/assets/38881179/142f3c33-27f5-42ad-8811-03e46e2c4665)

- Gradient가 특정 Threshold보다 클때 Gradient를 2-Norm으로 나누고 Threshold를 곱해준다.
- g^은 네트워크에서 사용하는 모든 매개변수의 기울기를 하나로 모은 것이다, 여러 가중치에 대한 편미분한 값을 행렬로써 들고 있는 것이다.


In [ ]:
import numpy as np
np.random.seed(3)

dW1 = np.random.rand(3, 3) * 10
dW2 = np.random.rand(3, 3) * 10
grads = [dW1, dW2]
max_norm = 5.0

def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    # max norm / total_norm 으로 비율을 구할 수 있다.
    # 1보다 크다면 total_norm이 max_norm보다 작은것
    # 1보다 작다면 total_norm이 max_norm보다 큰것
    
    if rate < 1:
        for grad in grads:
            grad *= rate
    # 크다면 각 원소에 그 비율만큼을 곱해준다.
    
N = 2
H = 3
T = 20

dh = np.ones((N, H))
Wh = np.random.randn(H, H)

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T)
    clip_grads(dh, 10.0)
    norm = np.sqrt(np.sum(dh**2)) / N
    norm_list.append(norm)

print(norm_list)
'''
output : [1.9910453959604886, 2.4999997535448575, 2.4999998146681337, 2.4999998110864134, 2.4999998085427713, 2.499999806860856, 2.4999998061738853, 2.4999998058007007, 2.499999805648307, 2.4999998055704595, 2.4999998055377968, 2.49999980552167, 2.4999998055147405, 2.4999998055113952, 2.499999805509932, 2.499999805509237, 2.4999998055089288, 2.4999998055087835, 2.4999998055087187, 2.4999998055086885]
- 기울기가 max_norm 이하로 유지 된다.
'''

